# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [2]:
wine = load_wine()
wine.keys()
df = pd.DataFrame(wine.data, columns=wine.feature_names).iloc[:, [9, 11, 12]]
x_tr, x_val, y_tr, y_val = train_test_split(df, wine.target, random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [4]:
skf = StratifiedKFold(n_splits=5, random_state=17, shuffle=True)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
lr = LogisticRegression(random_state=17, multi_class='ovr')
lr.fit(x_tr, y_tr)
scores_train_lr = cross_val_score(lr, x_tr, y_tr, cv=skf)
print("Средняя доля правильных ответов на обучении = {:.5f}".format(scores_train_lr.mean()))
scores_validate_lr = cross_val_score(lr, x_val, y_val, cv=skf)
print("Средняя доля правильных ответов на валидации = {:.5f}".format(scores_validate_lr.mean()))

Средняя доля правильных ответов на обучении = 0.90969
Средняя доля правильных ответов на валидации = 0.95556


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [7]:
from sklearn.svm import SVC

In [8]:
svc = SVC(random_state=17)
svc.fit(x_tr, y_tr)
scores_train_svc = cross_val_score(svc, x_tr, y_tr, cv=skf)
print("Средняя доля правильных ответов на обучении = {:.5f}".format(scores_train_svc.mean()))
scores_validate_svc = cross_val_score(svc, x_val, y_val, cv=skf)
print("Средняя доля правильных ответов на валидации = {:.5f}".format(scores_validate_svc.mean()))

Средняя доля правильных ответов на обучении = 0.69231
Средняя доля правильных ответов на валидации = 0.75556


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
confusion_matrix(y_val, lr.predict(x_val))

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 0,  4, 13]])

In [11]:
confusion_matrix(y_val, svc.predict(x_val))

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 2, 15,  0]])

### 5.
Для каждой модели выведите classification report.

In [12]:
print(classification_report(y_val, lr.predict(x_val)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45



In [13]:
print(classification_report(y_val, svc.predict(x_val)))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



/home/vladimir/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
